In [1]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from xgboost import XGBRegressor

In [2]:
final_predictions = []

In [3]:
# This code is from my first submission where I used only Ordinal Encoder on the categorical columns. I decided to use
# this also to ensemble.

df = pd.read_csv('../input/30-days-folds/train_folds.csv')
df_test = pd.read_csv('../input/30-days-of-ml/test.csv')
sample_submission = pd.read_csv('../input/30-days-of-ml/sample_submission.csv')

useful_features = [c for c in df.columns if c not in ("id", "target", "KFold")]
object_cols = [col for col in useful_features if 'cat' in col]
df_test = df_test[useful_features]

scores = []

for fold in range(5):
    xtrain = df[df.KFold != fold].reset_index(drop = True)
    xvalid = df[df.KFold == fold].reset_index(drop = True)
    xtest = df_test.copy()

    ytrain = xtrain.target
    yvalid = xvalid.target
    
    xtrain = xtrain[useful_features]
    xvalid = xvalid[useful_features]
    
    ordinal_encoder = preprocessing.OrdinalEncoder()
    xtrain[object_cols] = ordinal_encoder.fit_transform(xtrain[object_cols])
    xvalid[object_cols] = ordinal_encoder.transform(xvalid[object_cols])
    xtest[object_cols] = ordinal_encoder.transform(xtest[object_cols])
    
    model = XGBRegressor(random_state = fold, tree_method = 'gpu_hist', gpu_id = 0, predictor = "gpu_predictor")
    model.fit(xtrain, ytrain)
    preds_valid = model.predict(xvalid)
    test_preds = model.predict(xtest)
    final_predictions.append(test_preds)
    rmse = mean_squared_error(yvalid, preds_valid, squared = False)
    print(fold, rmse)
    scores.append(rmse)

print(np.mean(scores), np.std(scores))

0 0.7245705537554137
1 0.7242510333821858
2 0.7270667092065692
3 0.7268359229595335
4 0.7257178555909586
0.7256884149789322 0.0011430674400777338


In [4]:
# Ordinal Encoding of the object columns.

df = pd.read_csv('../input/30-days-folds/train_folds.csv')
df_test = pd.read_csv('../input/30-days-of-ml/test.csv')
sample_submission = pd.read_csv('../input/30-days-of-ml/sample_submission.csv')

useful_features = [c for c in df.columns if c not in ("id", "target", "KFold")]
object_cols = [col for col in useful_features if 'cat' in col]
df_test = df_test[useful_features]

scores = []

for fold in range(5):
    xtrain =  df[df.KFold != fold].reset_index(drop = True)
    xvalid = df[df.KFold == fold].reset_index(drop = True)
    xtest = df_test.copy()

    ytrain = xtrain.target
    yvalid = xvalid.target
    
    xtrain = xtrain[useful_features]
    xvalid = xvalid[useful_features]
    
    ordinal_encoder = preprocessing.OrdinalEncoder()
    xtrain[object_cols] = ordinal_encoder.fit_transform(xtrain[object_cols])
    xvalid[object_cols] = ordinal_encoder.transform(xvalid[object_cols])
    xtest[object_cols] = ordinal_encoder.transform(xtest[object_cols])
    
    model = XGBRegressor(random_state = fold, tree_method = 'gpu_hist', gpu_id = 0, predictor = "gpu_predictor")
    model.fit(xtrain, ytrain)
    preds_valid = model.predict(xvalid)
    test_preds = model.predict(xtest)
    final_predictions.append(test_preds)
    rmse = mean_squared_error(yvalid, preds_valid, squared = False)
    print(fold, rmse)
    scores.append(rmse)

print(np.mean(scores), np.std(scores))

0 0.7245705537554137
1 0.7242510333821858
2 0.7270667092065692
3 0.7268359229595335
4 0.7257178555909586
0.7256884149789322 0.0011430674400777338


In [5]:
xtrain.head()

,cat0,cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,cat9,...,cont4,cont5,cont6,cont7,cont8,cont9,cont10,cont11,cont12,cont13
0,1.0,1.0,1.0,2.0,1.0,1.0,0.0,4.0,2.0,13.0,...,0.610706,0.400361,0.160266,0.310921,0.389470,0.267559,0.237281,0.377873,0.322401,0.869850
1,1.0,1.0,0.0,0.0,1.0,3.0,0.0,5.0,0.0,14.0,...,0.276853,0.533087,0.558922,0.516294,0.594928,0.341439,0.906013,0.921701,0.261975,0.465083
2,1.0,1.0,0.0,2.0,1.0,3.0,0.0,4.0,2.0,10.0,...,0.284667,0.668980,0.239061,0.732948,0.679618,0.574844,0.346010,0.714610,0.540150,0.280682
3,0.0,0.0,0.0,2.0,1.0,3.0,0.0,4.0,0.0,13.0,...,0.287595,0.686964,0.420667,0.648182,0.684501,0.956692,1.000773,0.776742,0.625849,0.250823
4,1.0,0.0,0.0,0.0,1.0,3.0,0.0,4.0,2.0,5.0,...,0.281511,0.396705,0.273454,0.824573,0.656325,0.677114,0.808445,0.615973,0.631677,0.283561


In [6]:
# Standardization of the numerical features.

df = pd.read_csv('../input/30-days-folds/train_folds.csv')
df_test = pd.read_csv('../input/30-days-of-ml/test.csv')
sample_submission = pd.read_csv('../input/30-days-of-ml/sample_submission.csv')

useful_features = [c for c in df.columns if c not in ("id", "target", "KFold")]
object_cols = [col for col in useful_features if 'cat' in col]
numerical_cols = [col for col in useful_features if col.startswith("cont")]
df_test = df_test[useful_features]

scores = []

for fold in range(5):
    xtrain =  df[df.KFold != fold].reset_index(drop = True)
    xvalid = df[df.KFold == fold].reset_index(drop = True)
    xtest = df_test.copy()

    ytrain = xtrain.target
    yvalid = xvalid.target
    
    xtrain = xtrain[useful_features]
    xvalid = xvalid[useful_features]
    
    ordinal_encoder = preprocessing.OrdinalEncoder()
    xtrain[object_cols] = ordinal_encoder.fit_transform(xtrain[object_cols])
    xvalid[object_cols] = ordinal_encoder.transform(xvalid[object_cols])
    xtest[object_cols] = ordinal_encoder.transform(xtest[object_cols])
    
    scaler = preprocessing.StandardScaler()
    xtrain[numerical_cols] = scaler.fit_transform(xtrain[numerical_cols])
    xvalid[numerical_cols] = scaler.transform(xvalid[numerical_cols])
    xtest[numerical_cols] = scaler.transform(xtest[numerical_cols])
    
    model = XGBRegressor(random_state = fold, tree_method = 'gpu_hist', gpu_id = 0, predictor = "gpu_predictor")
    model.fit(xtrain, ytrain)
    preds_valid = model.predict(xvalid)
    test_preds = model.predict(xtest)
    final_predictions.append(test_preds)
    rmse = mean_squared_error(yvalid, preds_valid, squared = False)
    print(fold, rmse)
    scores.append(rmse)

print(np.mean(scores), np.std(scores))

0 0.7241755479182882
1 0.7241138968948254
2 0.7267386816038165
3 0.7268357864120136
4 0.725667388462628
0.7255062602583143 0.001185068397378747


In [7]:
xtrain.head()

,cat0,cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,cat9,...,cont4,cont5,cont6,cont7,cont8,cont9,cont10,cont11,cont12,cont13
0,1.0,1.0,1.0,2.0,1.0,1.0,0.0,4.0,2.0,13.0,...,0.494430,-0.482753,-1.461134,-1.037482,-0.453925,-0.951662,-0.928283,-0.429503,-0.696227,1.622629
1,1.0,1.0,0.0,0.0,1.0,3.0,0.0,5.0,0.0,14.0,...,-0.895099,0.098048,0.434116,-0.095141,0.403149,-0.612898,1.684568,2.020153,-0.940601,-0.192685
2,1.0,1.0,0.0,2.0,1.0,3.0,0.0,4.0,2.0,10.0,...,-0.862575,0.692711,-1.086533,0.898970,0.756434,0.457329,-0.503458,1.087321,0.184394,-1.019693
3,0.0,0.0,0.0,2.0,1.0,3.0,0.0,4.0,0.0,13.0,...,-0.850388,0.771411,-0.223159,0.510024,0.776804,2.208214,2.054810,1.367190,0.530979,-1.153603
4,1.0,0.0,0.0,0.0,1.0,3.0,0.0,4.0,2.0,5.0,...,-0.875711,-0.498752,-0.923024,1.319384,0.659266,0.926267,1.303352,0.643014,0.554549,-1.006782


In [8]:
# Log transformation of the numerical columns.
# This is genrally done to include the contributions of outliers as well.

df = pd.read_csv("../input/30-days-folds/train_folds.csv")
df_test = pd.read_csv("../input/30-days-of-ml/test.csv")
sample_submission = pd.read_csv("../input/30-days-of-ml/sample_submission.csv")

useful_features = [c for c in df.columns if c not in ("id", "target", "KFold")]
object_cols = [col for col in useful_features if 'cat' in col]
numerical_cols = [col for col in useful_features if col.startswith("cont")]
df_test = df_test[useful_features]

for col in numerical_cols:
    df[col] = np.log1p(df[col])
    df_test[col] = np.log1p(df_test[col])

scores = []

for fold in range(5):
    xtrain =  df[df.KFold != fold].reset_index(drop = True)
    xvalid = df[df.KFold == fold].reset_index(drop = True)
    xtest = df_test.copy()

    ytrain = xtrain.target
    yvalid = xvalid.target
    
    xtrain = xtrain[useful_features]
    xvalid = xvalid[useful_features]
    
    ordinal_encoder = preprocessing.OrdinalEncoder()
    xtrain[object_cols] = ordinal_encoder.fit_transform(xtrain[object_cols])
    xvalid[object_cols] = ordinal_encoder.transform(xvalid[object_cols])
    xtest[object_cols] = ordinal_encoder.transform(xtest[object_cols])
    
    model = XGBRegressor(random_state = fold, tree_method = 'gpu_hist', gpu_id = 0, predictor = "gpu_predictor")
    model.fit(xtrain, ytrain)
    preds_valid = model.predict(xvalid)
    test_preds = model.predict(xtest)
    final_predictions.append(test_preds)
    rmse = mean_squared_error(yvalid, preds_valid, squared = False)
    print(fold, rmse)
    scores.append(rmse)

print(np.mean(scores), np.std(scores))

0 0.7245867071148808
1 0.7242518770698644
2 0.7269464580617742
3 0.7267203050271116
4 0.7255892005274619
0.7256189095602186 0.001087249680887288


In [9]:
# Polynomial Features.
# Done so that our model looks if there is some kind of interaction between our columns.

df = pd.read_csv("../input/30-days-folds/train_folds.csv")
df_test = pd.read_csv("../input/30-days-of-ml/test.csv")
sample_submission = pd.read_csv("../input/30-days-of-ml/sample_submission.csv")

useful_features = [c for c in df.columns if c not in ("id", "target", "KFold")]
object_cols = [col for col in useful_features if 'cat' in col]
numerical_cols = [col for col in useful_features if col.startswith("cont")]
df_test = df_test[useful_features]

poly = preprocessing.PolynomialFeatures(degree = 3, interaction_only = True, include_bias = False)
train_poly = poly.fit_transform(df[numerical_cols])
test_poly = poly.fit_transform(df_test[numerical_cols])

df_poly = pd.DataFrame(train_poly, columns=[f"poly_{i}" for i in range(train_poly.shape[1])])
df_test_poly = pd.DataFrame(test_poly, columns=[f"poly_{i}" for i in range(test_poly.shape[1])])

df = pd.concat([df, df_poly], axis = 1)
df_test = pd.concat([df_test, df_test_poly], axis = 1)

useful_features = [c for c in df.columns if c not in ("id", "target", "KFold")]
object_cols = [col for col in useful_features if 'cat' in col]
df_test = df_test[useful_features]

scores = []

for fold in range(5):
    xtrain =  df[df.KFold != fold].reset_index(drop = True)
    xvalid = df[df.KFold == fold].reset_index(drop = True)
    xtest = df_test.copy()

    ytrain = xtrain.target
    yvalid = xvalid.target
    
    xtrain = xtrain[useful_features]
    xvalid = xvalid[useful_features]
    
    ordinal_encoder = preprocessing.OrdinalEncoder()
    xtrain[object_cols] = ordinal_encoder.fit_transform(xtrain[object_cols])
    xvalid[object_cols] = ordinal_encoder.transform(xvalid[object_cols])
    xtest[object_cols] = ordinal_encoder.transform(xtest[object_cols])
    
    model = XGBRegressor(random_state = fold, tree_method = 'gpu_hist', gpu_id = 0, predictor = "gpu_predictor")
    model.fit(xtrain, ytrain)
    preds_valid = model.predict(xvalid)
    test_preds = model.predict(xtest)
    ''' 
    final_predictions.append(test_preds) 
    Since this model gave a large rmse, removing it from the ensembled models. 
    '''
    rmse = mean_squared_error(yvalid, preds_valid, squared = False)
    print(fold, rmse)
    scores.append(rmse)

print(np.mean(scores), np.std(scores))

0 0.729073179900137
1 0.7286941123028183
2 0.7302315824391516
3 0.7304305210608322
4 0.7297044930462646
0.7296267777498406 0.0006624450323974544


In [10]:
train_poly

array([[ 0.20146955, -0.01482185,  0.66969879, ...,  0.07799239,
         0.06654311,  0.10597081],
       [ 0.74306801,  0.36741066,  1.02160538, ...,  0.38837833,
         0.11038887,  0.11230024],
       [ 0.74270829,  0.31038344, -0.01267284, ...,  0.35469623,
         0.30970906,  0.25648548],
       ...,
       [ 0.54427918,  0.06093709,  0.59095471, ...,  0.10226708,
         0.11599784,  0.11048749],
       [ 0.30006232,  0.61311817,  0.2852126 , ...,  0.05859871,
         0.03645857,  0.07716364],
       [ 0.85775185,  0.62852758,  0.53962466, ...,  0.24531827,
         0.32400103,  0.21361362]])

In [11]:
type(train_poly.shape)

tuple

In [12]:
df_poly

,poly_0,poly_1,poly_2,poly_3,poly_4,poly_5,poly_6,poly_7,poly_8,poly_9,...,poly_459,poly_460,poly_461,poly_462,poly_463,poly_464,poly_465,poly_466,poly_467,poly_468
0,0.201470,-0.014822,0.669699,0.136278,0.610706,0.400361,0.160266,0.310921,0.389470,0.267559,...,0.023990,0.020468,0.055224,0.032596,0.087945,0.075034,0.028907,0.077992,0.066543,0.105971
1,0.743068,0.367411,1.021605,0.365798,0.276853,0.533087,0.558922,0.516294,0.594928,0.341439,...,0.285127,0.081042,0.143873,0.082445,0.146364,0.041601,0.218769,0.388378,0.110389,0.112300
2,0.742708,0.310383,-0.012673,0.576957,0.285074,0.650609,0.375348,0.902567,0.555205,0.843531,...,0.391698,0.342018,0.482478,0.283242,0.399564,0.348886,0.251436,0.354696,0.309709,0.256485
3,0.429551,0.620998,0.577942,0.280610,0.284667,0.668980,0.239061,0.732948,0.679618,0.574844,...,0.142137,0.107437,0.055828,0.221888,0.115301,0.087152,0.133559,0.069402,0.052459,0.108342
4,1.058291,0.367492,-0.052389,0.232407,0.287595,0.686964,0.420667,0.648182,0.684501,0.956692,...,0.743678,0.599208,0.240146,0.465070,0.186388,0.150179,0.486499,0.194976,0.157099,0.121931
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
299995,0.697124,0.483452,0.297770,0.195821,0.307883,0.769792,0.450538,0.934360,1.005077,0.853726,...,0.383627,0.251679,0.182677,0.633433,0.459767,0.301630,0.313510,0.227556,0.149288,0.375733
299996,0.446200,0.715135,0.610931,0.601730,0.736713,0.528056,0.508502,0.358247,0.257825,0.433525,...,0.035032,0.075304,0.107479,0.067157,0.095851,0.206041,0.046630,0.066553,0.143063,0.127585
299997,0.544279,0.060937,0.590955,0.905308,0.277074,0.688747,0.372425,0.364936,0.383224,0.551825,...,0.229655,0.260490,0.089633,0.248115,0.085375,0.096838,0.297206,0.102267,0.115998,0.110487
299998,0.300062,0.613118,0.285213,0.406851,0.805963,0.344404,0.424243,0.382028,0.468819,0.351036,...,0.061953,0.038546,0.033657,0.081581,0.071235,0.044320,0.067109,0.058599,0.036459,0.077164


In [13]:
# One Hot Encoding of the model.

df = pd.read_csv("../input/30-days-folds/train_folds.csv")
df_test = pd.read_csv("../input/30-days-of-ml/test.csv")
sample_submission = pd.read_csv("../input/30-days-of-ml/sample_submission.csv")

useful_features = [c for c in df.columns if c not in ("id", "target", "KFold")]
object_cols = [col for col in useful_features if 'cat' in col]
df_test = df_test[useful_features]

scores = []

for fold in range(5):
    xtrain =  df[df.KFold != fold].reset_index(drop = True)
    xvalid = df[df.KFold == fold].reset_index(drop = True)
    xtest = df_test.copy()

    ytrain = xtrain.target
    yvalid = xvalid.target
    
    xtrain = xtrain[useful_features]
    xvalid = xvalid[useful_features]
    
    ohe = preprocessing.OneHotEncoder(sparse = False, handle_unknown = "ignore")
    xtrain_ohe = ohe.fit_transform(xtrain[object_cols])
    xvalid_ohe = ohe.transform(xvalid[object_cols])
    xtest_ohe = ohe.transform(xtest[object_cols])
        
    xtrain_ohe = pd.DataFrame(xtrain_ohe, columns = [f"ohe_{i}" for i in range(xtrain_ohe.shape[1])])
    xvalid_ohe = pd.DataFrame(xvalid_ohe, columns = [f"ohe_{i}" for i in range(xvalid_ohe.shape[1])])
    xtest_ohe = pd.DataFrame(xtest_ohe, columns = [f"ohe_{i}" for i in range(xtest_ohe.shape[1])])
    
    xtrain = pd.concat([xtrain, xtrain_ohe], axis = 1)
    xvalid = pd.concat([xvalid, xvalid_ohe], axis = 1)
    xtest = pd.concat([xtest, xtest_ohe], axis = 1)
    
    xtrain = xtrain.drop(object_cols, axis = 1)
    xvalid = xvalid.drop(object_cols, axis = 1)
    xtest = xtest.drop(object_cols, axis = 1)
    
    model = XGBRegressor(random_state = fold, tree_method = 'gpu_hist', gpu_id = 0, predictor = "gpu_predictor")
    model.fit(xtrain, ytrain)
    preds_valid = model.predict(xvalid)
    test_preds = model.predict(xtest)
    final_predictions.append(test_preds)
    rmse = mean_squared_error(yvalid, preds_valid, squared = False)
    print(fold, rmse)
    scores.append(rmse)

print(np.mean(scores), np.std(scores))

0 0.7244255014738967
1 0.7245139958781214
2 0.7264465446086561
3 0.7264028943362871
4 0.7257096926265366
0.7254997257846996 0.0008811227736574191


In [14]:
# One Hot Encoding of categorical variables and then standarization of OHE & numerical columns.

df = pd.read_csv("../input/30-days-folds/train_folds.csv")
df_test = pd.read_csv("../input/30-days-of-ml/test.csv")
sample_submission = pd.read_csv("../input/30-days-of-ml/sample_submission.csv")

useful_features = [c for c in df.columns if c not in ("id", "target", "KFold")]
object_cols = [col for col in useful_features if 'cat' in col]
df_test = df_test[useful_features]

scores = []

for fold in range(5):
    xtrain =  df[df.KFold != fold].reset_index(drop = True)
    xvalid = df[df.KFold == fold].reset_index(drop = True)
    xtest = df_test.copy()

    ytrain = xtrain.target
    yvalid = xvalid.target
    
    xtrain = xtrain[useful_features]
    xvalid = xvalid[useful_features]
    
    ohe = preprocessing.OneHotEncoder(sparse = False, handle_unknown = "ignore")
    xtrain_ohe = ohe.fit_transform(xtrain[object_cols])
    xvalid_ohe = ohe.transform(xvalid[object_cols])
    xtest_ohe = ohe.transform(xtest[object_cols])
        
    xtrain_ohe = pd.DataFrame(xtrain_ohe, columns = [f"ohe_{i}" for i in range(xtrain_ohe.shape[1])])
    xvalid_ohe = pd.DataFrame(xvalid_ohe, columns = [f"ohe_{i}" for i in range(xvalid_ohe.shape[1])])
    xtest_ohe = pd.DataFrame(xtest_ohe, columns = [f"ohe_{i}" for i in range(xtest_ohe.shape[1])])
    
    xtrain = pd.concat([xtrain, xtrain_ohe], axis = 1)
    xvalid = pd.concat([xvalid, xvalid_ohe], axis = 1)
    xtest = pd.concat([xtest, xtest_ohe], axis = 1)
    
    xtrain = xtrain.drop(object_cols, axis = 1)
    xvalid = xvalid.drop(object_cols, axis = 1)
    xtest = xtest.drop(object_cols, axis = 1)
    
    modified_features = [c for c in xtrain.columns]
    modified_object_cols = [col for col in modified_features if 'cat' in col]
    modified_numerical_cols = [col for col in modified_features if 'cat' not in col]

    scaler = preprocessing.StandardScaler()
    xtrain[modified_numerical_cols] = scaler.fit_transform(xtrain[modified_numerical_cols])
    xvalid[modified_numerical_cols] = scaler.transform(xvalid[modified_numerical_cols])
    xtest[modified_numerical_cols] = scaler.transform(xtest[modified_numerical_cols])
    
    model = XGBRegressor(random_state = fold, tree_method = 'gpu_hist', gpu_id = 0, predictor = "gpu_predictor")
    model.fit(xtrain, ytrain)
    preds_valid = model.predict(xvalid)
    test_preds = model.predict(xtest)
    final_predictions.append(test_preds)
    rmse = mean_squared_error(yvalid, preds_valid, squared = False)
    print(fold, rmse)
    scores.append(rmse)

print(np.mean(scores), np.std(scores))

0 0.7244324833714519
1 0.7245329062537117
2 0.7262739942272469
3 0.7264029729278617
4 0.7260672928259383
0.7255419299212421 0.0008720458471300098


## I tried to ensemble all of the models because they were giving as in almost the same errors.

In [15]:
print(final_predictions)

[array([8.12425 , 8.322003, 8.358131, ..., 8.204037, 8.193138, 7.95051 ],
      dtype=float32), array([7.991268 , 8.385105 , 8.321518 , ..., 8.241875 , 7.9872713,
       8.003579 ], dtype=float32), array([7.961162 , 8.332988 , 8.373009 , ..., 8.447979 , 8.130552 ,
       7.9890165], dtype=float32), array([7.943543, 8.310012, 8.400516, ..., 8.45448 , 8.116053, 7.888312],
      dtype=float32), array([8.024726, 8.312614, 8.400369, ..., 8.380058, 8.277814, 8.231191],
      dtype=float32), array([8.12425 , 8.322003, 8.358131, ..., 8.204037, 8.193138, 7.95051 ],
      dtype=float32), array([7.991268 , 8.385105 , 8.321518 , ..., 8.241875 , 7.9872713,
       8.003579 ], dtype=float32), array([7.961162 , 8.332988 , 8.373009 , ..., 8.447979 , 8.130552 ,
       7.9890165], dtype=float32), array([7.943543, 8.310012, 8.400516, ..., 8.45448 , 8.116053, 7.888312],
      dtype=float32), array([8.024726, 8.312614, 8.400369, ..., 8.380058, 8.277814, 8.231191],
      dtype=float32), array([8.079691, 8.33

In [16]:
np.column_stack(final_predictions).shape

(200000, 30)

In [17]:
predictions_submit = np.mean(np.column_stack(final_predictions), axis = 1)

In [18]:
sample_submission.target = predictions_submit
sample_submission.to_csv("submission.csv", index = False)

In [19]:
sample_submission

,id,target
0,0,8.018686
1,5,8.336782
2,15,8.386957
3,16,8.487537
4,17,8.146621
...,...,...
199995,499987,8.085788
199996,499990,8.357494
199997,499991,8.338929
199998,499994,8.114846
